<b>Scraping and Analyzing Fed Beige Books</b><br>
Project by Alexander Trentin<br>
<br>
Part 2: Analysis

In [4]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer 
from textblob import TextBlob

%matplotlib inline

In [5]:
#import the file which was created in Notebook 1 
df_original = pd.read_csv('fulltext.csv')

In [6]:
#The sectors are obsolete, as they are mapped now to general categories
#The descriptions should be combined for each category
#To do that, a 
df = df_original.groupby(['Date','District','Category'])['Description']\
    .apply(lambda x: " ".join(x))
df = df.reset_index()

In [7]:
#get rid of periods to separate sentences later easier
df.Description = df.Description.str.replace("U.S.","US")
df.Description = df.Description.str.replace("D.C.","DC")
df.Description = df.Description.str.replace("St.","St")
df.Description = df.Description.str.replace("Va.","Va")
df.Description = df.Description.str.replace("N.C.","NC")
df.Description = df.Description.str.replace("W.","W")
df.Description = df.Description.str.replace("S.C.","SC")
df.Description = df.Description.str.replace("S.D.","SD")
df.Description = df.Description.str.replace("Md.","Md")
df.Description = df.Description.str.replace(r"[0-9](\.)[0-9]",r",")

Text Analysis<br>
- use sklearn to get the most frequent phrases
- use a manual mapping file if a phrase is positive or negative, if it contains an adjective

In [8]:
#Add adverbs and adjectives into one dataframe
adverbs = pd.read_csv('conv.data.adv',index_col=False,names=['word'])
adjectives = pd.read_csv('conv.data.adj',index_col=False,names=['word'])

frames = [adverbs, adjectives]

adjectives = pd.concat(frames)
list_adjectives = list(adjectives.word)

In [9]:
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    words = blob.words
    return words

In [10]:
stop_words = ["the","district","was","were","next","previous","of","according"
              "beige","first","second","third","sector","since","our",
             "reported","noted","and","reports","reporting","period","over","outlook","york",
             "francisco","january","kansas","dallas","in","from","during","districts",
             "demand","commercial","for","contacts","continued",
             "compared","boston","a","as","at","activity","sector","ago","year","february","march","month",
             "creighton","recent","atlanta","chicago","ny","nj","louis","pa","managers",
             "university","omaha","to","months","report","minneapolis","indicated","that","according",
             "'s","one","two","three","four","five","six","seven","eight","nine","ten","which","percent",
             "survey","richmond","is","by","philadelphia","cleveland","20","7","major","year-to-date","this",
             "announced","on","eighth","cents","an","are","although","did","jersey","england","city",
             "year-ago", "dakota", "south", "carolina", "or", "but", "general", "generally", "may",
             "july","october","twelve","virginia","overall","siena","college","most",'been','year-over-year',
             'all','across','except','particularly','book']

vec = CountVectorizer(stop_words=stop_words, tokenizer = textblob_tokenizer, ngram_range=(2,2), min_df=90)

matrix = vec.fit_transform(df.Description)

results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
print(results.shape)
list_words = list(vec.get_feature_names())

(6195, 523)


In [219]:
vec = CountVectorizer(stop_words=stop_words, tokenizer = textblob_tokenizer, ngram_range=(3,3), min_df=23)

matrix = vec.fit_transform(df.Description)

results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
print(results.shape)
list_words = list_words + list(vec.get_feature_names())

(6136, 802)


In [220]:
vec = CountVectorizer(stop_words=stop_words, tokenizer = textblob_tokenizer, ngram_range=(4,4), min_df=13)

matrix = vec.fit_transform(df.Description)

results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
print(results.shape)
list_words = list_words + list(vec.get_feature_names())


(6136, 591)


In [221]:
vec = CountVectorizer(stop_words=stop_words, tokenizer = textblob_tokenizer, ngram_range=(5,5), min_df=8)

matrix = vec.fit_transform(df.Description)

results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
print(results.shape)
list_words = list_words + list(vec.get_feature_names())


(6136, 682)


In [264]:
list_words = set(list_words)
list_relevant = []
for word in list_words:
    for adjective in list_adjectives:
        for this_word in word.split(" "):
            if adjective == this_word:
                list_relevant.append(word)

In [11]:
mapping = pd.read_csv("map_adjectives.csv")

In [12]:
vocab_list = list(mapping.adjective)
#count words according to updated adjective list
vec = CountVectorizer(stop_words=stop_words, tokenizer = textblob_tokenizer, ngram_range=(1,3), vocabulary=vocab_list)

matrix = vec.fit_transform(df.Description)

results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
print(results.shape)

(6195, 105)


In [13]:
veryNegativeList = list(mapping[mapping.score == -2].adjective)
negativeList = list(mapping[mapping.score == -1].adjective)
neutralList = list(mapping[mapping.score == 0].adjective)
positiveList = list(mapping[mapping.score == 1].adjective)
veryPositiveList = list(mapping[mapping.score == 2].adjective)

In [14]:
df['veryNegative'] = results[veryNegativeList].sum(axis=1)
df['negative'] = results[negativeList].sum(axis=1)
df['neutral'] = results[neutralList].sum(axis=1)
df['positive'] = results[positiveList].sum(axis=1)
df['veryPositive'] = results[veryPositiveList].sum(axis=1)

In [15]:
# add for each combination of Date, District and Category
# the most important phrases and the economc sentiment

def compare_phrases(row):
    
    adjective_score = row['veryNegative'] * (-2) + row['negative'] * (-1) + row['positive']*1 + row['veryPositive'] * 2
    adjective_score_counter = row['veryNegative'] + row['negative'] + row ['neutral'] + row['positive'] + row['veryPositive']
    
    if adjective_score_counter > 0:
        adjective_score = adjective_score / adjective_score_counter
    else:
        adjective_score = np.nan
    
    this_description = row['Description']

    #split up the descriptions by sentences
    this_description_sentences = this_description.split('.')
    
    #the maximum proportion of adjectives found in a phrase
    description_score_max = 0
    
    #this variable will be later filled with the most expressive sentence
    important_sentence = ""

    #Each sentence in the description is split up into words

    #Check if a word is in the adjective mapping table
    #Count how many words are in each sentence
    
    for description in this_description_sentences:
        description_words = description.split(' ')
        len_description = len(description_words)
        
        description_score = 0
        
        for adjective in list(mapping.adjective):
            if adjective in description:
                description_score += 1
        
        description_score = description_score / len_description
        
        
        #Check if the sentence has the highest score regarding adjectives
        #If it has the highest, it is for the time being the most important sentence
                
        if description_score > description_score_max:
            description_score_max = description_score
            important_sentence = description

  
    # return the sentiment score and the most important sentence
    
    return pd.Series({
        'adjective_score' : adjective_score,
        'important_phrase' : important_sentence
    })

In [16]:
df_new = df[['Date','District','Category']]
df_new = df.apply(compare_phrases, axis=1).join(df_new)

In [21]:
df_new

,adjective_score,important_phrase,Date,District,Category
0,-1.000000,\nGulf Coast crude oil inventories declined fu...,200801,Atlanta,"Agriculture, Energy, Natural Resources"
1,-0.500000,"Manufacturing varied by industry, with defen...",200801,Atlanta,Economy
2,0.800000,Input costs continue to increase for goods o...,200801,Atlanta,"Employment, Prices, Wages"
3,-0.200000,Credit card lending was a notable exception ...,200801,Atlanta,Finance
4,0.666667,"However, producers of residential housing-re...",200801,Atlanta,"Manufacturing, Services"
5,-0.600000,Residential construction was significantly b...,200801,Atlanta,"Real Estate, Construction"
6,1.000000,\nReports concerning tourism continued to be q...,200801,Atlanta,"Retail, Consumer Spending"
7,-0.428571,"Closing out 2007 with mixed results, business ...",200801,Boston,Economy
8,1.000000,Sales of aircraft equipment and pharmaceutica...,200801,Boston,"Manufacturing, Services"
9,-0.086957,"\nIn most markets and sectors, rents and vaca...",200801,Boston,"Real Estate, Construction"


In [22]:
#remove white space in "important phrases"
df_new['important_phrase'] = df_new['important_phrase'].astype(str).str.strip()

In [24]:
#add a column color according to the sentiment (adjective score)
#starting from very positive until very negative
df_new['color'] = '#0571b0'
df_new.loc[df_new['adjective_score']<1, 'color'] = '#92c5de'
df_new.loc[df_new['adjective_score']<0.4, 'color'] = '#B8B8B8'
df_new.loc[df_new['adjective_score']<-0.3, 'color'] = '#f4a582'
df_new.loc[df_new['adjective_score']<-0.9, 'color'] = '#ca0020'

In [25]:
#mapping dictionary from the district names to codes used in the URL
report_districts = {
    "National Summary":"su",
    "Atlanta":"at",
    "Boston":"bo",
    "Chicago":"ch",
    "Cleveland":"cl",
    "Dallas":"da",
    "Kansas City":"kc",
    "Minneapolis":"mi",
    "New York":"ny",
    "Philadelphia":"ph",
    "Richmond":"ri",
    "San Francisco":"sf",
    "St. Louis":"sl"
}

In [26]:
#Create an URL column out of the date and the district code
#Later to be used as a link in the map
def getURLreport(row):
    this_date = str(row['Date'])[0:4]+"-"+str(row['Date'])[4:6]
    this_district = report_districts[row['District']]
    this_URL = "https://www.minneapolisfed.org/news-and-events/beige-book-archive/"+this_date+"-"+this_district
    return pd.Series({'link_URL':this_URL})

df_new = df_new.apply(getURLreport, axis=1).join(df_new)

In [27]:
report_dates = [
"200801",
"200803",
"200804",
"200806",
"200807",
"200809",
"200810",
"200812",
"200901",
"200903",
"200904",
"200906",
"200907",
"200909",
"200910",
"200912",
"201001",
"201003",
"201004",
"201006",
"201007",
"201009",
"201010",
"201012",
"201101",
"201103",
"201104",
"201106",
"201107",
"201109",
"201110",
"201111",
"201201",
"201202",
"201204",
"201206",
"201207",
"201208",
"201210",
"201211",
"201301",
"201303",
"201304",
"201306",
"201307",
"201309",
"201310",
"201312",
"201401",
"201403",
"201404",
"201406",
"201407",
"201409",
"201410",
"201412",
"201501",
"201503",
"201504",
"201506",
"201509",
"201510",
"201512",
"201601",
"201603",
"201604",
"201606",
"201607",
"201609",
"201610",
"201611",
"201701",
"201703",
"201704",
"201705",
"201707"
]
for report_date in report_dates:
    df_new[df_new.Date == int(report_date)].to_csv('data_'+report_date+'.csv', index=False)